# Final Assignment: Use of Python and Classical Mechanics to code a game similar to "Angry Birds" #

## Introduction ##
The aim of this notebook is to create a game, using programming knowledge and skills acquired in addition to knowledge of classical mechanics, that is similar to the popular mobile game called "Angry Birds". Throughout this notebook, I will refer to the sphere as "bird" only because this game is quite similar to Angry Birds. The game will work by the user inputting an initial velocity, in meters per second (m/s), and an initial angle, in degrees. Then "bird", represented by a red sphere, is launched at the initial velocity at the angle specified and it follows a parabolic trajectory. If and when the "bird" hits a blue target, that is generated at random x positions, the momentum of the "bird" at the point of impact is calculated and hence the applied torque on the target is also calculated. The restoring torque of the target, which is constant for a given mass of the target, is also worked out. Determination of whether the target topples or not is dependent on the magnitude of each of the torques calculated. Should the "bird" miss the target, the game should run again such that a new initial velocity and angle are inputted to launch the "bird" once more to see if it topples the target. This loop will continue until the target has been toppled. I will carry out this task by first importing the relevant modules and functions and setting up the canvas on which the animations will take place, then I shall define a function that will contain a while loop with all the code for the animations and the mechanics of the game. Then finally the function will be called, allowing the game to commence. It is important to note that the momentum of the "bird" is also the impulse.

It was noticed that when the "bird" missed the target, the label did not show until the new velocity and angle had been entered. This error could not be corrected in any way and is most likely due to the program being used to write this code.

## Importing Functions ##

In [ ]:
# importing required modules and functions
import numpy as np
from vpython import sphere, color, rate, canvas, vector, curve, label, box, cross, mag, random

The above code cell is where I have imported the relevant module, such as Numpy, and certain functions from the module called VPython were imported, that were required to code this game. These are modules that are additional to the standard Python functions.

## Setting Up The Canvas ##

In [ ]:
# setting up the vpython canvas
scene = canvas(width=640, height=480, center=vector(8,0,0),range=8)
ground = curve(pos=[(0,0,0),(16,0,0)],color=color.green)

# initial "bird" coordinates
x0 = 0.0
y0 = 0.0

# additional parameters
t0 = 0.0 # initial time, s
g = 9.81 # gravitational acceleration, m/s2
dt = 0.01 # time interval for loop animation, in seconds
m1 = 0.1 # mass of the "bird", kg
m2 = 50 # mass of the target, kg
r = 0.05 # radius of "bird", m

# setting the scene on the canvas
x_target = (10*random())+5 # random x-coordinate between 5 and 15 for target
bird = sphere(pos = vector(x0,y0,0),radius = 0.3, make_trail=True, color=color.red) # sphere representing the "bird"
target = box(pos=vector(x_target,1,0), color=color.blue, length=0.5, height=2, width=0.5) # box representing the target
hit_tolerance = r + target.width/2 # sum of "bird" radius and half the width of target
t = t0 # initialising time
y = y0 # initialising y value
x = x0 # initialising x value

Then the canvas was set up using the "canvas()" and "curve()" functions which created a blank canvas and green line to represent the ground. Then the initial coordinates of the bird are chosen, and in this case it is the origin as that is what is stated in the specification. There are additional parameters that are also stated that are very useful for this task, such as the gravitational acceleration of the Earth, the pre-selected time interval, the radius of the bird that was specifically chosen to be used in calculations and the masses of both the "bird" and the target. The mass of the "bird" was provided in the assignment script, however the mass of the target was not, therefore an appropriate mass was assigned to the target.

The "bird" and target were then created using the "sphere()" and the "box()" respectively, and I have written code to randomly generate a number between 5 and 15, shown on line 22, as stated in the specification and it has been assigned to "x_target". This is used to generate a target at a different x position each time the code is run. The "bird" is placed at the origin of the canvas, with a radius of 0.3 for the purpose of the animation even though the radius used in the calculations is different. The "hit_tolerance", on line 25, is equal to half the width of the target, 0.25 m, plus the radius of the "bird", 0.05 m, and it is used to determine whether a collision has taken place. If the distance between the x position of the "bird" and the target is less than or equal to the "hit_tolerance" then a collision between the "bird" and target has taken place.

Furthermore the animation will run below the code cell above as the code for creating the canvas is written in the code cell above, therefore the animation is automatically played in that same canvas.

## Playing the Game ##

In [ ]:
# defining while loop as function to call it later
def angrybirds():
    x = x0 # initialising x value
    y = y0 # initialising y value
    t = t0 # initialising time
    bird.pos = vector(x0,y0,0) # setting position of bird to origin
    
    # input of initial angle and speed
    dtheta = float(input("Input the initial angle in degrees: "))
    theta = np.radians(dtheta) # converts angle to radians
    v0 = float(input("Input the initial speed in metres/second: "))
    
    # start of animation loop
    while x_target - x > hit_tolerance and y >= 0.0: # condition for "bird" about to launch or in air
        rate(40) # sets rate of animation
        x = x0 + v0 * t * np.cos(theta) # calculates new x position
        y = y0 + v0 * t * np.sin(theta) - (0.5 * g * t**2) # calculates new y-position
        bird.pos = vector(x,y,0) # puts "bird" in new position
        t = t + dt # proceeds to next value of time
        if x_target - x <= hit_tolerance and 0.0 < y <= 2.0: # condition for "bird" hitting target
            px = m1 * v0 * np.cos(theta) # calculates momentum of "bird" in x-direction
            py = m1 * v0 * np.sin(theta) - m1 * g * t # calculates momentum of "bird" in y-direction
            tor_restore = (-m2 * g * target.width/2) # restoring torque of target
            F_applied = vector(px, py, 0)/dt # force applied by "bird" on target
            da = vector(-target.width, y, 0) # vector from rotation point to impact point
            tor_applied = cross(F_applied, da) # torque applied on target
            label(pos=vector(8,6,0), text='Well done! You hit the target!')
            # print statements for the required data to be outputted
            print("\nThe height of the impact point is:", y, "m")
            print("The bird's momentum at the point of impact is:", mag(vector(px, py, 0)), "kgm/s")
            print("The magnitude of the restoring torque is:", abs(tor_restore), "Nm")
            print("The applied torque is:", mag(tor_applied), "Nm")
            if mag(tor_applied) > abs(tor_restore): # condition for target toppling
                label(pos=vector(8,5,0), text='The target has been toppled!')
                label(pos=vector(8,4,0), text='You win!')
            elif mag(tor_applied) <= abs(tor_restore): # condition for target not toppling
                label(pos=vector(8,5,0), text='The target has not been toppled! Try Again...')
                angrybirds() # calling the function once target is not toppled
        # if statement for the "bird" missing target
        if x_target - x > hit_tolerance and y <= 0.0 and x > 0 or x_target - x <= hit_tolerance and y > 2.0:
            label(pos=vector(8,6,0), text='Aw no! You missed the target! Try again.')
            angrybirds() # calling the function once target is missed
angrybirds()

The above code cell contains the most important code for this game as it is the main code for the animations and the calculations. All of the code above has been defined as a function of "angrybirds()", therefore when the function is run on its own, as can be seen on line 43, the whole defined function above will be run. Initially I have set the "x", "y" and "t" values to 0 as this will be useful when the function is run again automatically due to the target not toppling or the "bird" missing the target. Then the position of the "bird" is reset to the origin so that it can be launched with a new initial velocity and angle. The inputs for angle and velocity have also been included, in lines 9 to 11, within the defined function so that when the "bird" does not hit the target or does not topple the target, then new values for velocity and angle can be inputted so that a new trajectory can be created for the "bird" to allow it to hit the target.

The main part of the program is of course the while loop, which will be used for moving the "bird" and determining whether the target has been hit and then toppled, in addition to restarting the game when the target is missed or not toppled so that the player can once again launch the "bird". The first line of the while loop is used for choosing the conditions that need to be met in order for the loop to start, and in this case the distance between the "bird" and the target has to be greater than the "hit_tolerance" and the vertical height of the "bird" above the ground has to be greater than or equal to 0, which represents the area within which the "bird" can move. Lines 16 to 19 are where the projectile motion of the "bird" is calculated, by first calculating the new x-position of the "bird" for the time, t, using the equation $ x=x_0+v_0tcos(\theta) $ and then the y-position is calculated with the equation $ y=y_0+v_0tsin(\theta)-0.5gt^2 $, which has an extra part due to the gravitational acceleration that affects the height. Then the "bird" is moved to this new position in line 18. Finally in line 19, the time is increased by the time interval, so it is 0.01 s bigger. This loop is run repeatedly until the initial condition of the while loop is no longer met or another condition is met.

Lines 20 to 32 contain an if statement that is only run when the distance between the "bird" and the target is less than or equal to the "hit_tolerance", which indicates that the target has been hit, and if the vertical height of the "bird" is above the ground but below or equal to the height of the target. These 2 conditions would account for any situation in which the target is hit. Therefore once the target is hit, the momentum of the "bird" in the x-direction is calculated, shown on line 21, using the equation $ p_x=m_1v_0cos(\theta) $ and the momentum in the y-direction is calculated using the equation $ p_y=m_1v_0sin(\theta)-m_1gt $, where  $ m_1 $ is the mass of the "bird". These are calculated because they will later be used to calculate the torque applied by the "bird" on the target. The restoring torque, which is the torque that will try to restore the target to its stable position when it is hit by the "bird", is then calculated using the equation $ |\tau_{restoring}|=-0.5m_2g\omega $, where $ m_2 $ is the mass of the target, that is a simplied version of $ \tau_{restoring}=\mathbf{F}_{grav}\times\mathbf{d}_r $ in which $ \mathbf{F}=m\mathbf{g} $ and $ \mathbf{d}_r=0.5\omega $. The $ \omega $ is the width of the target, as shown on line 23. Lines 24 to 26 contain the code to calculate the torque applied on the target, which is done by using the equation $ \tau_{applied}=\mathbf{F}_{applied}\times\mathbf{d}_a $, where $ \mathbf{F}_{applied} $ is the vector momentum of the ball at the point of impact divided by the time of collision, which is 0.01 s, and $ \mathbf{d}_a $ is the vector from the point of rotation of the target to the point of impact on the target. The vector $ \mathbf{d}_a $ was calculated and is shown on line 25, then the cross product of the $ \mathbf{F}_{applied} $ and the $ \mathbf{d}_a $ is done on line 26. This has now calculated the torque applied on the target when the "bird" collides with it, so this value together with the value for the restoring torque can be used to determine whether the target has been toppled, but first print statements and labels have been used to output relevant calculated data and inform the player of their progress in the game. On lines 27 to 32 both a label, to inform the player that the target has been successfully hit, and several print statements have been included, which give information such as the height of the impact point, the momentum at the point of impact, the applied torque and the restoring torque.

The next if and elif statements, from line 33 to 38, are used to decide whether the target will topple or not. The target will only topple if the magnitude of the applied torque is greater than the magnitude of the restoring torque, this is because the torque applied will be too great for the restoring torque to balance the target hence it will topple. I have chosen to include these if and elif statements inside a previous if statement because they do not need to be run unless the "bird" has hit the target, so it does not make sense to place it anywhere else, as it would show an error since there would be no variables with the names "tor_applied" and "tor_restore". When the target is determined to be toppled, a label is printed on the canvas informing the player of this, as well as another label to tell the player that they have won. However if the target is shown to have not toppled, a different label is shown that informs the player that it hasn't toppled and the function is called again as shown on line 38. This is because if the target does not topple, then the player must once again choose a new launch angle and velocity to launch the "bird" once more. This was the main reason for defining the whole loop as a function, as it allows the function to be included in the loop itself.

The final if statement starting on line 40 is used for when the player launches the "bird" but then misses the target by either going above or below the target, which is reflected in the conditions chosen for the if statement to be run. Should the player miss the target, then a label on the canvas will inform the player of this and the game will rerun so that the "bird" can be launched once more.

## Improvements ##

In reality the "bird" would experience air resistance after it is launched, which has not been accounted for in this game, therefore an equation for the drag force experienced by the bird can be included to decrease the change in both x and y directions, as velocities in both directions would decrease. This would also decrease the momentum of the "bird" in both the x and y directions as the momentum would be transferred to the air molecules. The drag force experienced could be modelled as being constant, however it would be more accurate to model it so that the drag force increases as the velocity of the "bird" increases. This would result in mechanics of the game being more accurate in terms of physics. This can be implemented by first calculating the velocity of the "bird" in the x and y directions, then using this velocity to calculate the drag force on the "bird" in each direction. From this the deceleration of the "bird" can be calculated, which is then used in suvat equations to apply the effects of air resistance to the position, velocity and momentum of the "bird".

Furthermore the rotation of the ball has not been taken into account. When a ball is launched through the air, it is very likely to rotate meaning it has angular momentum. This can affect the distance moved in both x and y directions, in addition to affecting the torque applied on the target.

Reducing the value of dt from 0.01 s to a lower value would make the values of height, momentum and applied torque more accurate as the loop is repeated for a shorter time interval so the moment the "bird" hits the target will be much more precise in time.

## Conclusion ##

As shown above, I have successfully coded a game similar to "Angry Birds" with my programming skills and knowledge of classical mechanics. Even though it is not as complex as "Angry Birds", it follows the basic principles of the game in which an object is launch at a target, which has to be toppled. The physics used is accurate, however basic it may be, but it can be improved to add to the realism of the game. Additionally the animations can be improved, such as by animating the target to topple when it is hit or to move when it is hit.